Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (50000, 28, 28) (50000,)
Validation set (2500, 28, 28) (2500,)
Test set (2500, 28, 28) (2500,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (50000, 784) (50000, 10)
Validation set (2500, 784) (2500, 10)
Test set (2500, 784) (2500, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

#### L2 Regularized Logistic Regression

Define graph

In [5]:
batch_size = 128
regularization = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) +\
    regularization/2*tf.nn.l2_loss(weights)
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Fit model

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 15.871655
Minibatch accuracy: 13.3%
Validation accuracy: 17.0%
Minibatch loss at step 500: 2.403866
Minibatch accuracy: 75.8%
Validation accuracy: 77.2%
Minibatch loss at step 1000: 1.761634
Minibatch accuracy: 80.5%
Validation accuracy: 77.2%
Minibatch loss at step 1500: 1.316510
Minibatch accuracy: 78.9%
Validation accuracy: 78.6%
Minibatch loss at step 2000: 1.114374
Minibatch accuracy: 78.9%
Validation accuracy: 79.8%
Minibatch loss at step 2500: 0.890684
Minibatch accuracy: 82.8%
Validation accuracy: 80.3%
Minibatch loss at step 3000: 0.699390
Minibatch accuracy: 83.6%
Validation accuracy: 80.5%
Test accuracy: 88.7%


#### Regularized Neural Network

Define graph

In [7]:
batch_size = 128
num_hidden_units = 1024
regularization = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables of input layer
  weights_input = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_units]))
  biases_input = tf.Variable(tf.zeros([num_hidden_units]))
    
  # Variables.
  weights_hidden = tf.Variable(
    tf.truncated_normal([num_hidden_units, num_labels]))
  biases_hidden = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  relus = tf.nn.relu(tf.matmul(tf_train_dataset, weights_input) + biases_input)
  logits_hidden = tf.matmul(relus, weights_hidden) + biases_hidden
  loss = tf.reduce_mean(c
    tf.nn.softmax_cross_entropy_with_logits(logits_hidden, tf_train_labels)) +\
    0.5*regularization*(tf.nn.l2_loss(weights_input) + tf.nn.l2_loss(weights_hidden))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_hidden)
  valid_relus = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_input) + biases_input)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_relus, weights_hidden) + biases_hidden)
  test_relus = tf.nn.relu(tf.matmul(tf_test_dataset, weights_input) + biases_input)
  test_prediction = tf.nn.softmax(
    tf.matmul(test_relus, weights_hidden) + biases_hidden)

Fit model

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1994.226929
Minibatch accuracy: 7.0%
Validation accuracy: 38.2%
Minibatch loss at step 500: 126.779457
Minibatch accuracy: 82.8%
Validation accuracy: 82.3%
Minibatch loss at step 1000: 10.902067
Minibatch accuracy: 87.5%
Validation accuracy: 86.3%
Minibatch loss at step 1500: 1.527885
Minibatch accuracy: 83.6%
Validation accuracy: 85.2%
Minibatch loss at step 2000: 0.745085
Minibatch accuracy: 85.9%
Validation accuracy: 85.9%
Minibatch loss at step 2500: 0.592083
Minibatch accuracy: 87.5%
Validation accuracy: 85.7%
Minibatch loss at step 3000: 0.496000
Minibatch accuracy: 88.3%
Validation accuracy: 85.0%
Test accuracy: 92.7%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [18]:
batch_size = 128
num_hidden_units = 1024
regularization = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables of input layer
  weights_input = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_units]))
  biases_input = tf.Variable(tf.zeros([num_hidden_units]))
    
  # Variables.
  weights_hidden = tf.Variable(
    tf.truncated_normal([num_hidden_units, num_labels]))
  biases_hidden = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  relus = tf.nn.relu(tf.matmul(tf_train_dataset, weights_input) + biases_input)
  logits_hidden = tf.matmul(relus, weights_hidden) + biases_hidden
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_hidden, tf_train_labels)) +\
    0.5*regularization*(tf.nn.l2_loss(weights_input) + tf.nn.l2_loss(weights_hidden))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_hidden)
  valid_relus = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_input) + biases_input)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_relus, weights_hidden) + biases_hidden)
  test_relus = tf.nn.relu(tf.matmul(tf_test_dataset, weights_input) + biases_input)
  test_prediction = tf.nn.softmax(
    tf.matmul(test_relus, weights_hidden) + biases_hidden)

In [19]:
num_steps = 3001
# Restricting train size
ix_small = range(int(batch_size*5.5))
train_dataset_small = train_dataset[ix_small,:]
train_labels_small = train_labels[ix_small,:]

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_small[offset:(offset + batch_size), :]
    batch_labels = train_labels_small[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1865.283447
Minibatch accuracy: 12.5%
Validation accuracy: 27.0%
Minibatch loss at step 500: 128.414093
Minibatch accuracy: 99.2%
Validation accuracy: 76.9%
Minibatch loss at step 1000: 10.586967
Minibatch accuracy: 99.2%
Validation accuracy: 79.4%
Minibatch loss at step 1500: 1.037373
Minibatch accuracy: 100.0%
Validation accuracy: 79.6%
Minibatch loss at step 2000: 0.250749
Minibatch accuracy: 100.0%
Validation accuracy: 79.3%
Minibatch loss at step 2500: 0.184268
Minibatch accuracy: 100.0%
Validation accuracy: 79.6%
Minibatch loss at step 3000: 0.177927
Minibatch accuracy: 100.0%
Validation accuracy: 79.5%
Test accuracy: 87.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

##### Regularization and dropout with whole training set

In [26]:
batch_size = 128
num_hidden_units = 1024
regularization = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables of input layer
  weights_input = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_units]))
  biases_input = tf.Variable(tf.zeros([num_hidden_units]))
    
  # Variables.
  weights_hidden = tf.Variable(
    tf.truncated_normal([num_hidden_units, num_labels]))
  biases_hidden = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  relus = tf.nn.relu(tf.matmul(tf_train_dataset, weights_input) + biases_input)
  logits_hidden = tf.nn.dropout(tf.matmul(relus, weights_hidden) + biases_hidden, keep_prob = 0.5) # Added dropout
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_hidden, tf_train_labels)) +\
    0.5*regularization*(tf.nn.l2_loss(weights_input) + tf.nn.l2_loss(weights_hidden))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_hidden)
  valid_relus = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_input) + biases_input)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_relus, weights_hidden) + biases_hidden)
  test_relus = tf.nn.relu(tf.matmul(tf_test_dataset, weights_input) + biases_input)
  test_prediction = tf.nn.softmax(
    tf.matmul(test_relus, weights_hidden) + biases_hidden)

In [27]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2068.532471
Minibatch accuracy: 12.5%
Validation accuracy: 18.0%
Minibatch loss at step 500: 127.644135
Minibatch accuracy: 52.3%
Validation accuracy: 80.4%
Minibatch loss at step 1000: 11.601990
Minibatch accuracy: 53.9%
Validation accuracy: 84.0%
Minibatch loss at step 1500: 2.138979
Minibatch accuracy: 55.5%
Validation accuracy: 84.4%
Minibatch loss at step 2000: 1.327963
Minibatch accuracy: 54.7%
Validation accuracy: 83.4%
Minibatch loss at step 2500: 1.303870
Minibatch accuracy: 53.1%
Validation accuracy: 84.2%
Minibatch loss at step 3000: 1.165913
Minibatch accuracy: 53.9%
Validation accuracy: 83.6%
Test accuracy: 91.6%


##### Regularization and dropout with small training set to show dropout properties

In [26]:
batch_size = 128
num_hidden_units = 1024
regularization = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables of input layer
  weights_input = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_units]))
  biases_input = tf.Variable(tf.zeros([num_hidden_units]))
    
  # Variables.
  weights_hidden = tf.Variable(
    tf.truncated_normal([num_hidden_units, num_labels]))
  biases_hidden = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  relus = tf.nn.relu(tf.matmul(tf_train_dataset, weights_input) + biases_input)
  logits_hidden = tf.nn.dropout(tf.matmul(relus, weights_hidden) + biases_hidden, keep_prob = 0.5) # Added dropout
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_hidden, tf_train_labels)) +\
    0.5*regularization*(tf.nn.l2_loss(weights_input) + tf.nn.l2_loss(weights_hidden))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_hidden)
  valid_relus = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_input) + biases_input)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_relus, weights_hidden) + biases_hidden)
  test_relus = tf.nn.relu(tf.matmul(tf_test_dataset, weights_input) + biases_input)
  test_prediction = tf.nn.softmax(
    tf.matmul(test_relus, weights_hidden) + biases_hidden)

In [28]:
num_steps = 3001
# Restricting train size
ix_small = range(int(batch_size*5.5))
train_dataset_small = train_dataset[ix_small,:]
train_labels_small = train_labels[ix_small,:]

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_small[offset:(offset + batch_size), :]
    batch_labels = train_labels_small[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2137.075684
Minibatch accuracy: 3.9%
Validation accuracy: 25.3%
Minibatch loss at step 500: 130.069702
Minibatch accuracy: 60.9%
Validation accuracy: 77.5%
Minibatch loss at step 1000: 11.461935
Minibatch accuracy: 57.8%
Validation accuracy: 79.1%
Minibatch loss at step 1500: 1.922180
Minibatch accuracy: 53.9%
Validation accuracy: 79.9%
Minibatch loss at step 2000: 1.079390
Minibatch accuracy: 59.4%
Validation accuracy: 79.8%
Minibatch loss at step 2500: 1.036792
Minibatch accuracy: 59.4%
Validation accuracy: 79.8%
Minibatch loss at step 3000: 1.093117
Minibatch accuracy: 53.9%
Validation accuracy: 80.0%
Test accuracy: 88.2%


##### Conclusion

With dropout there is not a huge amount of overfitting even if the training set is small.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [89]:
batch_size = 512
num_hidden_units_L1 = 400
num_hidden_units_L2 = 400
regularization = 0.04

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # count the number of steps taken.
    global_step = tf.Variable(0)
    
    # Variables of input layer
    weights_input = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_hidden_units_L1]))
    biases_input = tf.Variable(tf.zeros([num_hidden_units_L1]))

    # Variables of HL1
    weights_hidden_L1 = tf.Variable(
        tf.truncated_normal([num_hidden_units_L1, num_hidden_units_L2]))
    biases_hidden_L1 = tf.Variable(tf.zeros([num_hidden_units_L2]))

    # Variables of HL2
    weights_hidden_L2 = tf.Variable(
        tf.truncated_normal([num_hidden_units_L2, num_labels]))
    biases_hidden_L2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    relus_L1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_input) + biases_input)
    relus_L2 = tf.nn.relu(tf.matmul(relus_L1, weights_hidden_L1) + biases_hidden_L1)
    #logits_output = tf.nn.dropout(tf.matmul(relus_L2, weights_hidden_L2) + biases_hidden_L2, keep_prob = 0.5)
    logits_output = tf.matmul(relus_L2, weights_hidden_L2) + biases_hidden_L2

    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits_output, tf_train_labels)) +\
        0.5*regularization*(tf.nn.l2_loss(weights_input) + tf.nn.l2_loss(weights_hidden_L1) + tf.nn.l2_loss(weights_hidden_L2))

    # Optimizer.
    
    #learning_rate = tf.train.exponential_decay(0.01, global_step, 10, np.e)
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits_output)
    
    valid_relus_L1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_input) + biases_input)
    valid_relus_L2 = tf.nn.relu(tf.matmul(valid_relus_L1, weights_hidden_L1) + biases_hidden_L1)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_relus_L2, weights_hidden_L2) + biases_hidden_L2)
    
    test_relus_L1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_input) + biases_input)
    test_relus_L2 = tf.nn.relu(tf.matmul(test_relus_L1, weights_hidden_L1) + biases_hidden_L1)
    test_prediction = tf.nn.softmax(tf.matmul(test_relus_L2, weights_hidden_L2) + biases_hidden_L2)

In [91]:
num_steps = 6001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print("Step: %d" % (step))
            print("Minibatch loss: %f" % l)
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Step: 0
Minibatch loss: 6527.588379
Minibatch accuracy: 7.0%
Validation accuracy: 15.8%
Step: 100
Minibatch loss: 3806.230469
Minibatch accuracy: 40.2%
Validation accuracy: 37.6%
Step: 200
Minibatch loss: 2762.354980
Minibatch accuracy: 45.1%
Validation accuracy: 42.6%
Step: 300
Minibatch loss: 2005.235229
Minibatch accuracy: 54.9%
Validation accuracy: 48.8%
Step: 400
Minibatch loss: 1455.803955
Minibatch accuracy: 59.0%
Validation accuracy: 55.3%
Step: 500
Minibatch loss: 1057.000854
Minibatch accuracy: 66.8%
Validation accuracy: 63.2%
Step: 600
Minibatch loss: 767.430359
Minibatch accuracy: 70.1%
Validation accuracy: 67.9%
Step: 700
Minibatch loss: 557.353638
Minibatch accuracy: 69.9%
Validation accuracy: 72.3%
Step: 800
Minibatch loss: 404.786835
Minibatch accuracy: 74.8%
Validation accuracy: 74.5%
Step: 900
Minibatch loss: 294.046295
Minibatch accuracy: 78.5%
Validation accuracy: 76.3%
Step: 1000
Minibatch loss: 213.683075
Minibatch accuracy: 79.9%
Validation accuracy: 

In [65]:
tf.train.exponential_decay?